In [15]:
from sklearn.datasets import fetch_20newsgroups
news_data = fetch_20newsgroups(subset='all',random_state=156)

In [16]:
print(news_data.keys())

dict_keys(['data', 'filenames', 'target_names', 'target', 'DESCR'])


In [29]:
pd.Series(news_data.target).value_counts().sort_index()

0     799
1     973
2     985
3     982
4     963
5     988
6     975
7     990
8     996
9     994
10    999
11    991
12    984
13    990
14    987
15    997
16    910
17    940
18    775
19    628
dtype: int64

In [18]:
import pandas as pd
print(pd.Series(news_data.target).value_counts().sort_index())
print(news_data.target_names)

0     799
1     973
2     985
3     982
4     963
5     988
6     975
7     990
8     996
9     994
10    999
11    991
12    984
13    990
14    987
15    997
16    910
17    940
18    775
19    628
dtype: int64
['alt.atheism', 'comp.graphics', 'comp.os.ms-windows.misc', 'comp.sys.ibm.pc.hardware', 'comp.sys.mac.hardware', 'comp.windows.x', 'misc.forsale', 'rec.autos', 'rec.motorcycles', 'rec.sport.baseball', 'rec.sport.hockey', 'sci.crypt', 'sci.electronics', 'sci.med', 'sci.space', 'soc.religion.christian', 'talk.politics.guns', 'talk.politics.mideast', 'talk.politics.misc', 'talk.religion.misc']


In [40]:
train_news = fetch_20newsgroups(subset = 'train',remove=('headers','footers','quotes'),random_state=156)
X_train = train_news.data
y_train = train_news.target
test_news = fetch_20newsgroups(subset = 'test',remove=('headers','footers','quotes'),random_state=156)
X_test = test_news.data
y_test = test_news.target

In [41]:
from sklearn.feature_extraction.text import CountVectorizer
cnt_vect = CountVectorizer()
cnt_vect.fit(X_train)
X_train_cnt_vect = cnt_vect.transform(X_train)

# 학습 데이터로 fit( )된 CountVectorizer를 이용하여 테스트 데이터를 feature extraction 변환 수행. 
X_test_cnt_vect = cnt_vect.transform(X_test)

print('학습 데이터 Text의 CountVectorizer Shape:',X_train_cnt_vect.shape)

학습 데이터 Text의 CountVectorizer Shape: (11314, 101631)


In [42]:
print('학습 데이터 크기 {0} , 테스트 데이터 크기 {1}'.format(len(train_news.data) , len(test_news.data)))
X_train_cnt_vect.shape
X_test_cnt_vect.shape

학습 데이터 크기 11314 , 테스트 데이터 크기 7532


(7532, 101631)

In [43]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import warnings
warnings.filterwarnings('ignore')

In [44]:
lr_clf = LogisticRegression(solver = 'liblinear')
lr_clf.fit(X_train_cnt_vect, y_train)
pred = lr_clf.predict(X_test_cnt_vect)
print(accuracy_score(y_test, pred))

0.616436537440255


In [45]:
from sklearn.feature_extraction.text import TfidfVectorizer

# TF-IDF Vectorization 적용하여 학습 데이터셋과 테스트 데이터 셋 변환. 
tfidf_vect = TfidfVectorizer()
tfidf_vect.fit(X_train)
X_train_tfidf_vect = tfidf_vect.transform(X_train)
X_test_tfidf_vect = tfidf_vect.transform(X_test)

# LogisticRegression을 이용하여 학습/예측/평가 수행. 
lr_clf = LogisticRegression(solver='liblinear')
lr_clf.fit(X_train_tfidf_vect , y_train)
pred = lr_clf.predict(X_test_tfidf_vect)
print('TF-IDF Logistic Regression 의 예측 정확도는 {0:.3f}'.format(accuracy_score(y_test ,pred)))

TF-IDF Logistic Regression 의 예측 정확도는 0.678


In [49]:
tfidf_vect = TfidfVectorizer(stop_words='english', ngram_range=(1,2), max_df=300)
tfidf_vect.fit(X_train)
X_train_tfidf_vect = tfidf_vect.transform(X_train)
X_test_tfidf_vect = tfidf_vect.transform(X_test)

lr_clf = LogisticRegression(solver='liblinear')
lr_clf.fit(X_train_tfidf_vect , y_train)
pred = lr_clf.predict(X_test_tfidf_vect)
print('TF-IDF Vectorized Logistic Regression 의 예측 정확도는 {0:.3f}'.format(accuracy_score(y_test ,pred)))

TF-IDF Vectorized Logistic Regression 의 예측 정확도는 0.690


In [52]:
from sklearn.model_selection import GridSearchCV
params = {'C' : [0.01, 0.1, 1, 5, 10]}
grid_cv_lr = GridSearchCV(lr_clf,param_grid=params, cv=3, scoring = 'accuracy',verbose=1)
grid_cv_lr.fit(X_train_tfidf_vect,y_train)
print(grid_cv_lr.best_params_)
pred = grid_cv_lr.predict(X_test_tfidf_vect)
print(accuracy_score(y_test, pred))

Fitting 3 folds for each of 5 candidates, totalling 15 fits
{'C': 10}
0.7039298990971854


In [56]:
from sklearn.pipeline import Pipeline
pipeline = Pipeline([
    ('tfidf_vect',TfidfVectorizer(stop_words = 'english')),
    ('lr_clf',LogisticRegression(solver='liblinear',C=10))
])
params = {'tfidf_vect__ngram_range':[(1,1),(1,2),(1,3)],
          'tfidf_vect__max_df':[100,300,700],
          'lr_clf__C':[1,5,10]}
grid_cv_pipe = GridSearchCV(pipeline,param_grid = params, cv = 3, scoring = 'accuracy',verbose=1)
grid_cv_pipe.fit(X_train, y_train)
print(grid_cv_pipe.best_params_,grid_cv_pipe.best_score_)
pred = grid_cv_pipe.predict(X_test)
print(accuracy_score(y_test,pred))

Fitting 3 folds for each of 27 candidates, totalling 81 fits


KeyboardInterrupt: 